<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MyBert_paragraph_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

('TPU address is', 'grpc://10.4.54.194:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5559332680567153430),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10445438877619348536),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11447852806683110462),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3560235416749192272),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2304354183769936225),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 10919000243704167055),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 6670514842439782462),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 13017166376088599804),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 539119

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
# import optimization
# import run_classifier
from bert import run_classifier_with_tfhub
# import tokenization

# import tfhub 
import tensorflow_hub as hub

W0520 19:37:19.440768 140680220788608 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
BUCKET = 'bert_example' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/smallBERT_ParWithLabel_seq128'.format(BUCKET)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL


***** Model output directory: gs://bert_example/bert-tfhub/models/smallBERT_ParWithLabel_seq128 *****


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" ##Small Bert
# # BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1" ##Big Bert
# def create_tokenizer_from_hub_module():
#   """Get the vocab file and casing info from the Hub module."""
#   with tf.Graph().as_default():
#     bert_module = hub.Module(BERT_MODEL_HUB)
#     tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
#     with tf.Session() as sess:
#       vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
#                                             tokenization_info["do_lower_case"]])
      
#   return bert.tokenization.FullTokenizer(
#       vocab_file=vocab_file, do_lower_case=do_lower_case)

# tokenizer = create_tokenizer_from_hub_module()

path = 'gs://bert_example/bert/uncased_L-12_H-768_A-12/vocab_tgt.txt'
f_in = tf.gfile.GFile('gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt')
f_out = tf.gfile.GFile(path,'w')
lines = f_in.readlines()


lines[1] = 'tgt\n'
for line in lines:
  f_out.write(line)
f_out.close()

VOCAB_FILE = os.path.join('gs://bert_example/bert/uncased_L-12_H-768_A-12', 'vocab_tgt.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
PREDICT_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 200
SAVE_SUMMARY_STEPS = 100




In [23]:
from tensorflow import keras
import os
import re

# directory = '/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv'
# data_train = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv', encoding='latin-1')
# data_dev = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_dev_v3.csv', encoding='latin-1')
# data_test = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_random_test_v3.csv', encoding='latin-1')
# data_test_fixed = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_fixed_test_v3.csv', encoding='latin-1')

# data_train = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_train_v3.csv', encoding='latin-1')
# data_dev = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_dev_v3.csv', encoding='latin-1')
# data_test = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_random_test_v3.csv', encoding='latin-1')
# data_test_fixed = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_fixed_test_v3.csv', encoding='latin-1')

def fix_doc_tgt(doc):

  new_doc = doc.replace('.[TGT]',' .\n[TGT] ')
  new_doc = new_doc.replace('\[TGT\]','tgt')
  return new_doc


def load_paragraphs(df):
    paragraph_labels = []
    paragraph_texts = []
    paragraph_ids = []
    doc_label = []
    num_doc = 0
    index = -1
    print(len(df['MASKED_DOCUMENT']))
    for doc in df['MASKED_DOCUMENT']:
      index += 1
      docs = doc.split('\n')
      doc_length = len(docs)

      if pd.isnull(df['Paragraph0'].iloc[index]):
#         Comment the following lines to skip documents with no paragraph labels, I can change it to distant supervision but I don't like :D
        paragraph_labels.append(df['TRUE_SENTIMENT'].iloc[index])
        paragraph_texts.append(doc.replace('\[TGT\]','tgt'))
        paragraph_ids.append(index)
        doc_label.append(df['TRUE_SENTIMENT'].iloc[index])
        num_doc +=1
        continue
      try:
        if  doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
          doc = fix_doc_tgt(doc)
          docs = doc.split('\n')
          doc_length = len(docs)
          if  doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
            print('error on document %d'% df['DOCUMENT_INDEX'].iloc[index])
            print('document length is %s'%str(doc_length))
            continue
      except Exception as e:
        print('error occured: %s'%str(e))
        print('this document has %d paragraphs %d' %(doc_length,df['DOCUMENT_INDEX'].iloc[index]))
        
      has_null_label = False
      for i in range(doc_length):  
        if pd.isnull(df['Paragraph%s'%str(i)].iloc[index]):
            print('document %d has null paragraph labels in index: %d'%(index,i))
            has_null_label = True
            break
            
      if has_null_label:
        continue

      for i in range(doc_length):
        
        
        paragraph_texts.append(docs[i])
        label_i = df['Paragraph%d'%i].iloc[index]
        paragraph_labels.append(label_i)
        paragraph_ids.append(index)
        doc_label.append(df['TRUE_SENTIMENT'].iloc[index])
    print('number of 1 paragraph docs: %d'%num_doc)
    return(paragraph_texts,paragraph_labels,paragraph_ids,doc_label)
  

# train_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv')
# train_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_train_v3.csv')

train_par_file = open('/content/train_v5.csv')
train_par_df = pd.read_csv(train_par_file)

# dev_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_dev_v3.csv')
# dev_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_dev_v3.csv')

dev_par_file = open('/content/dev_v5.csv')
dev_par_df = pd.read_csv(dev_par_file)

# test_random_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_random_test_v3.csv')
# test_random_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_random_test_v3.csv')

test_random_par_file = open('/content/random_test_v5.csv')
test_random_par_df = pd.read_csv(test_random_par_file)

# test_fixed_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_fixed_test_v3.csv')
# test_fixed_par_file = open('/content/alldata_3Dec_7Dec_PS_reindex_enclosed_masked_fixed_test_v3.csv')

test_fixed_par_file =  open('/content/fixed_test_v5.csv')
test_fixed_par_df = pd.read_csv(test_fixed_par_file)


# Load all files from a directory in a DataFrame.
def load_paragraphs_file(df):
  data = {}
  (paragraphs,labels,paragraph_ids,doc_label ) = load_paragraphs(df)
  
  data["sentence"] = paragraphs
  data["sentiment"] =labels
  data["doc_id"] = paragraph_ids
  data['doc_label'] = doc_label
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset_par(df,index = None):
  df_new = load_paragraphs_file(df)
  pos_df = df_new[df_new['sentiment'] == 'Positive']
  neg_df = df_new[df_new['sentiment'] == 'Negative']
  neu_df = df_new[df_new['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  
  shuffled_out = pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)
  
  return (shuffled_out,df_new)

print('-------processing train set ---------')

train_par,train_ori = load_dataset_par(train_par_df)
print('-------processing dev set ---------')

dev_par,dev_ori = load_dataset_par(dev_par_df)
print('-------processing random test set ---------')

test_par,test_ori = load_dataset_par(test_random_par_df)
print('-------processing fixed test set ---------')

test_fixed_par,test_fixed_ori = load_dataset_par(test_fixed_par_df)
print('Number of train paragraphs %d, Number of dev paragraphs %d,  Number of fixed test paragraphs %d,  Number of random test paragraphs %d, '
      %(len(train_par),len(dev_par),len(test_par),len(test_fixed_par)))


-------processing train set ---------
1501
number of 1 paragraph docs: 252


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

-------processing dev set ---------
284
number of 1 paragraph docs: 45
-------processing random test set ---------
285
document 267 has null paragraph labels in index: 9
document 267 has null paragraph labels in index: 10
number of 1 paragraph docs: 52
-------processing fixed test set ---------
340
error on document 2171
document length is 6
number of 1 paragraph docs: 114
Number of train paragraphs 9715, Number of dev paragraphs 1926,  Number of fixed test paragraphs 1876,  Number of random test paragraphs 1925, 


In [24]:
print(len(set(test_ori['doc_id'])))


284


In [0]:
def set_polarity(row):
  if row['sentiment'] == 'Positive':
    return 1
  if row['sentiment'] == 'Negative':
    return -1
  if row['sentiment'] == 'Neutral':
    return 0
  
train_ori['polarity'] = train_ori.apply(lambda row: set_polarity(row),axis=1)
dev_ori['polarity'] = dev_ori.apply(lambda row: set_polarity(row),axis=1)
test_ori['polarity'] = test_ori.apply(lambda row: set_polarity(row),axis=1)
test_fixed_ori['polarity'] = test_fixed_ori.apply(lambda row: set_polarity(row),axis=1)

In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [-1, 0, 1]


In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
train_InputExamples_par = train_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
train_features = run_classifier.convert_examples_to_features(train_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)


dev_InputExamples_par = dev_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
dev_features = run_classifier.convert_examples_to_features(dev_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)


test_InputExamples_par = test_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
test_features = run_classifier.convert_examples_to_features(test_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)


test_InputExamples_fixed_par = test_fixed_par.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_features_fixed = run_classifier.convert_examples_to_features(test_InputExamples_fixed_par, label_list, MAX_SEQ_LENGTH, tokenizer)


train_ori_InputExample = train_ori.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
train_ori_features = run_classifier.convert_examples_to_features(train_ori_InputExample, label_list, MAX_SEQ_LENGTH, tokenizer)


dev_ori_InputExample = dev_ori.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
dev_ori_features = run_classifier.convert_examples_to_features(dev_ori_InputExample, label_list, MAX_SEQ_LENGTH, tokenizer)


test_ori_InputExample = test_ori.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_ori_features = run_classifier.convert_examples_to_features(test_ori_InputExample, label_list, MAX_SEQ_LENGTH, tokenizer)



test_fixed_ori_InputExample =test_fixed_ori.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_fixed_ori_features = run_classifier.convert_examples_to_features(test_fixed_ori_InputExample, label_list, MAX_SEQ_LENGTH, tokenizer)


# train_par_features = bert.run_classifier.convert_examples_to_features(train_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)
# dev_par_features = bert.run_classifier.convert_examples_to_features(dev_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)
# test_par_features = bert.run_classifier.convert_examples_to_features(test_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)
# test_par_features_fixed = bert.run_classifier.convert_examples_to_features(test_InputExamples_fixed_par, label_list, MAX_SEQ_LENGTH, tokenizer)



num_train_steps = int(len(train_InputExamples_par) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
ITERATIONS_PER_LOOP = int(len(train_InputExamples_par) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [0]:
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities})
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)



In [0]:
# Train the model
def model_train(estimator):
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_InputExamples_par, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_InputExamples_par)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [0]:
# tf.logging.set_verbosity(tf.logging.FATAL)
# model_train(estimator_from_tfhub)

#Evaluation and Prediction 

In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = dev_InputExamples_par#processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
# model_eval(estimator_from_tfhub)

In [0]:
#   eval_accuracy = 0.5473958
#   eval_loss = 1.1722633
#   global_step = 909
#   loss = 1.2637016
  
  
#   ***** Eval results *****
#   eval_accuracy = 0.5375
#   eval_loss = 1.2887536
#   global_step = 1212
#   loss = 1.3013718
def model_predict(estimator,prediction_examples,input_features,checkpoint_path=None):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  if not checkpoint_path == None: 
    predictions = estimator.predict(predict_input_fn,checkpoint_path=checkpoint_path)
  else:
    predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities']) for sentence, prediction in zip(prediction_examples, predictions)]



In [39]:
import numpy as np
from sklearn import metrics
##### Document Level Predictions:
tf.logging.set_verbosity(tf.logging.FATAL)
def predict_doc_class_by_paragraph_pred(predictions, dev_pred_df,method='avg'):

    dev_pred_df['predictions'] = pd.Series([item[1] for item in predictions])
    doc_ids = set(dev_pred_df['doc_id'])
    document_predicted_class = []
    document_true_class = []
    for doc_id in doc_ids:
#       print('processing document: ', doc_id)
      neg = []
      neu = []
      pos = []
      labels = []
      true_label = list(dev_pred_df[dev_pred_df['doc_id']==doc_id]['doc_label'])[0]
      
      probabilities = list(dev_pred_df[dev_pred_df['doc_id']==doc_id]['predictions'])
      if pd.isnull(probabilities).any():
          print('null paragraph found %d'%doc_id)
          continue
      for pr in probabilities:

        neg.append(pr[0])
        neu.append(pr[1])
        pos.append(pr[2])
        labels.append(np.argmax(pr))
      all_class_pres = [np.average(neg),np.average(neu),np.average(pos)]
      if method == 'avg' or method == 'both':
        predicted_class= np.argmax(all_class_pres)
      if method == 'majority' or method == 'both':
        predicted_class = max(labels, key=labels.count)
      document_true_class.append(true_label)
      if predicted_class == 0:
        document_predicted_class.append('Negative')
      elif predicted_class == 1:
        document_predicted_class.append('Neutral')
      elif predicted_class == 2:
        document_predicted_class.append('Positive')    
      else:
        document_predicted_class.append(None)
    print(metrics.confusion_matrix(y_pred=document_predicted_class,y_true=document_true_class))
    print(metrics.classification_report(y_pred=document_predicted_class,y_true = document_true_class))

ckpt = 4030
####### Previous:
### 1212: avg: 46 majority: 44
### 4030: avg: 49 majority: 46
### 4545: avg: 49 majority: 47 -> fixed: avg: 37 maj: 37 , random:  avg: 32 maj: 31
### 4210: avg: 43 majority: 43
### 7375: avg: 44 majority: 45 -> majo: fixed: 29, random: 32
 
####### NEW ( all paragraphs)
### 1212: avg: 49 majority: 47 
### 4030: avg: 54 majority: 52 -> fixed: avg: 43, majority: 44 , random: avg: 46, majority: 45 -> reported in paper
### 4545: avg: 54 majority: 52 -> fixed: avg: 42, majority: 41 , random: avg: 48, majority: 47
### 4210: avg: 42 majority: 43
### 7375: avg: 44 majority: 45 -> 

####### NEW ( labeled paragraphs)
### 1212: avg: 46 majority: 44 
### 4030: avg: 52 majority: 49  
### 4545: avg: 52 majority: 50 -> 
### 4210: avg: 41 majority: 42 
### 7375: avg: 43 majority: 43 -> 

predictions = model_predict(estimator_from_tfhub,dev_ori_InputExample,dev_ori_features,checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)
predict_doc_class_by_paragraph_pred(predictions,dev_ori,method='avg')
predict_doc_class_by_paragraph_pred(predictions,dev_pred_df = dev_ori,method='majority')


predictions = model_predict(estimator_from_tfhub,test_ori_InputExample,test_ori_features,checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)
predict_doc_class_by_paragraph_pred(predictions,test_ori,method='avg')
predict_doc_class_by_paragraph_pred(predictions,test_ori,method='majority')


predictions = model_predict(estimator_from_tfhub,test_fixed_ori_InputExample,test_fixed_ori_features,checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)
predict_doc_class_by_paragraph_pred(predictions,test_fixed_ori,method='avg')
predict_doc_class_by_paragraph_pred(predictions,test_fixed_ori,method='majority')


null paragraph found 283
[[  7   9   7]
 [  3  51  43]
 [  1  44 118]]
              precision    recall  f1-score   support

    Negative       0.64      0.30      0.41        23
     Neutral       0.49      0.53      0.51        97
    Positive       0.70      0.72      0.71       163

   micro avg       0.62      0.62      0.62       283
   macro avg       0.61      0.52      0.54       283
weighted avg       0.62      0.62      0.62       283

null paragraph found 283
[[  7   8   8]
 [  4  49  44]
 [  1  51 111]]
              precision    recall  f1-score   support

    Negative       0.58      0.30      0.40        23
     Neutral       0.45      0.51      0.48        97
    Positive       0.68      0.68      0.68       163

   micro avg       0.59      0.59      0.59       283
   macro avg       0.57      0.50      0.52       283
weighted avg       0.60      0.59      0.59       283

[[  7  10  11]
 [  3  42  42]
 [  4  61 104]]
              precision    recall  f1-score   supp

In [47]:
import numpy as np
from sklearn import metrics
predictions = model_predict(estimator_from_tfhub,dev_InputExamples_par[:-2],dev_features[:-2],checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)

labels = ["Negative","Neutral", "Positive"]
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(dev_par['sentiment'])[:-2]

print('---------------Paragraph-level Dev results-------------')
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

predictions = model_predict(estimator_from_tfhub,test_InputExamples_par,test_features,checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)

labels = ["Negative","Neutral", "Positive"]
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_par['sentiment'])
print('---------------Paragraph-level Random Test results-------------')
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))


predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_par[:-1],test_features_fixed[:-1],checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-%d'%ckpt)

labels = ["Negative","Neutral", "Positive"]
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed_par['sentiment'])[:-1]
print('---------------Paragraph-level Fixed Test results-------------')
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

print('---------------Document-level results-------------')
# doc_labels = []
# pred_labels = []
# for doc_id in dev_par['doc_id'].unique():
#   doc_labels.append(list(dev_par[dev_par['doc_id']==doc_id]['doc_label'])[0])
#   ids = dev_par.index[dev_par['doc_id']==doc_id].tolist()
#   par_labels = []
#   for idd in ids :
#     par_labels.append(labels_val[idd])
#   mod_label = max(set(par_labels), key=par_labels.count)
#   pred_labels.append(mod_label)
# dev_par

# print(metrics.confusion_matrix(y_pred=pred_labels,y_true=doc_labels))
# print(metrics.classification_report(y_pred=pred_labels,y_true = doc_labels))


# print(len(doc_labels))
# print(len(pred_labels))



---------------Paragraph-level Dev results-------------
[[ 33  69  51]
 [ 58 360 354]
 [ 41 348 610]]
              precision    recall  f1-score   support

    Negative       0.25      0.22      0.23       153
     Neutral       0.46      0.47      0.46       772
    Positive       0.60      0.61      0.61       999

   micro avg       0.52      0.52      0.52      1924
   macro avg       0.44      0.43      0.43      1924
weighted avg       0.52      0.52      0.52      1924

---------------Paragraph-level Random Test results-------------
[[ 33  81  65]
 [ 65 325 320]
 [ 50 365 572]]
              precision    recall  f1-score   support

    Negative       0.22      0.18      0.20       179
     Neutral       0.42      0.46      0.44       710
    Positive       0.60      0.58      0.59       987

   micro avg       0.50      0.50      0.50      1876
   macro avg       0.41      0.41      0.41      1876
weighted avg       0.50      0.50      0.49      1876

---------------Paragraph-l

In [0]:
_predictions = model_predict(estimator_from_tfhub,test_InputExamples_fixed_par[:1724],checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-4030')
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_fixed_par['sentiment'])[:1724]




In [35]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[ 31  89 117]
 [ 57 270 353]
 [ 56 290 461]]
              precision    recall  f1-score   support

    Negative       0.22      0.13      0.16       237
     Neutral       0.42      0.40      0.41       680
    Positive       0.50      0.57      0.53       807

   micro avg       0.44      0.44      0.44      1724
   macro avg       0.38      0.37      0.37      1724
weighted avg       0.43      0.44      0.43      1724



In [0]:
predictions = model_predict(estimator_from_tfhub,test_InputExamples_par[:1780],checkpoint_path = 'gs://bert_example/bert-tfhub/models/smallBERT-15It_allParagraphs/model.ckpt-4030')
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
true_label = list(test_par['sentiment'])[:1780]


In [39]:
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[ 25  77  57]
 [ 62 307 306]
 [ 46 352 548]]
              precision    recall  f1-score   support

    Negative       0.19      0.16      0.17       159
     Neutral       0.42      0.45      0.44       675
    Positive       0.60      0.58      0.59       946

   micro avg       0.49      0.49      0.49      1780
   macro avg       0.40      0.40      0.40      1780
weighted avg       0.49      0.49      0.49      1780



In [0]:
# Save Prediction on Files
predictions = model_predict(estimator_from_tfhub,train_ori_InputExample)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
train_ori['prediction'] = pd.Series(labels_val)
train_ori.to_csv(open('train_prediction.csv','w'))

predictions = model_predict(estimator_from_tfhub,dev_ori_InputExample)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
dev_ori['prediction'] = pd.Series(labels_val)
dev_ori.to_csv(open('dev_prediction.csv','w'))

predictions = model_predict(estimator_from_tfhub,test_ori_InputExample)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
test_ori['prediction'] = pd.Series(labels_val)
test_ori.to_csv(open('test_prediction.csv','w'))

predictions = model_predict(estimator_from_tfhub,test_fixed_ori_InputExample)
labels_val = []
for item in predictions:
  labels_val.append(labels[np.argmax(item[1])])
test_fixed_ori['prediction'] = pd.Series(labels_val)
test_fixed_ori.to_csv(open('test_fixed_prediction.csv','w'))